In [19]:
import mysql.connector
from mysql.connector import Error
import json
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()  # take environment variables


True

In [20]:
# loading the mysql database credentials
host = os.getenv("host")
user = os.getenv("user")
password = os.getenv("password")
database = os.getenv("database")
port = os.getenv("port")

In [21]:
port

'3306'

In [22]:

def connect_to_database():
    """Establish connection to MySQL database"""
    try:
        connection = mysql.connector.connect(
            host=host,
            database=database,
            user=user,  # Update with your credentials
            password=password
        )
        return connection
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def fetch_all_candidates():
    """Fetch all candidate data from joined tables"""
    connection = connect_to_database()
    if not connection:
        return None
    
    try:
        cursor = connection.cursor(dictionary=True)
        query = '''
        SELECT ur.resumeid, p.name, p.location, p.email, p.phone,
               e.degree, e.major, e.school, e.startdate, e.enddate, e.grade,
               we.company, we.role, we.startdate AS work_startdate, we.enddate AS work_enddate, 
               GROUP_CONCAT(s.skillname) as skills, we.description, 
               we.locationcity, we.locationcountry, ur.ocrtext, 
               mu.name AS user_name, mu.notes, mu.preferredrole, mu.workavailability, 
               mu.fulltimestatus, mu.fulltimeavailability, mu.fulltimesalary,
               mu.fulltimesalarycurrency, mu.parttimesalary, mu.summary
        FROM UserResume ur
        JOIN Education e ON e.educationId = ur.resumeId
        JOIN WorkExperience we ON we.workExperienceId = ur.resumeId
        JOIN PersonalInformation p ON p.personalInfromationId = ur.resumeId
        JOIN mystartup_users mu ON mu.userid = ur.resumeid
        JOIN mystartupUserSkills ms ON ms.userId = ur.resumeid
        JOIN skills s ON s.skillid = ms.skillid
        GROUP BY ur.resumeid
        ORDER BY ur.resumeid
        '''
        cursor.execute(query)
        records = cursor.fetchall()
        return records
    except Error as e:
        print(f"Error fetching data: {e}")
        return None
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()


In [23]:
# Test Phase 1
print("Phase 1: Testing database connection and data fetching")
candidates_data = fetch_all_candidates()

Phase 1: Testing database connection and data fetching


In [24]:
len(candidates_data)

5

In [25]:
if candidates_data:
    print(f"total candidate in database:- {len(candidates_data)}")
    print("Sample candidate:", candidates_data[0]['name'] if candidates_data else "No data")

total candidate in database:- 5
Sample candidate: Rahul Sharma


### Profile-summary Generation

In [53]:
def create_candidate_profile_text(candidate):
    """Create comprehensive profile text for embedding generation"""
    profile_parts = []
    
    try:
        # Personal and Role Information
        profile_parts.append(f"Name: {candidate['name']}")
        profile_parts.append(f"Preferred Role: {candidate['preferredrole']}")
        
        # Parse location JSON
        if isinstance(candidate['location'], str):
            location = json.loads(candidate['location'])
        else:
            location = candidate['location']
        profile_parts.append(f"Location: {location.get('city', '')}, {location.get('state', '')}, {location.get('country', '')}")
        
        # Education Background
        profile_parts.append(f"Education: {candidate['degree']} in {candidate['major']} from {candidate['school']}")
        profile_parts.append(f"Academic Grade: {candidate['grade']}")
        
        # Work Experience
        profile_parts.append(f"Current/Recent Role: {candidate['role']} at {candidate['company']}")
        profile_parts.append(f"Work Location: {candidate['locationcity']}, {candidate['locationcountry']}")
        profile_parts.append(f"Work Description: {candidate['description']}")
        
        # Skills
        profile_parts.append(f"Technical Skills: {candidate['skills']}")
        
        # Availability and Compensation
        profile_parts.append(f"Work Availability: {candidate['workavailability']}")
        profile_parts.append(f"Full-time Status: {candidate['fulltimestatus']}")
        profile_parts.append(f"Expected Salary: {candidate['fulltimesalary']} {candidate['fulltimesalarycurrency']}")
        
        # Resume OCR and Summary
        profile_parts.append(f"Resume Content: {candidate['ocrtext']}")
        profile_parts.append(f"Professional Summary: {candidate['summary']}")
        profile_parts.append(f"Additional Notes: {candidate['notes']}")
        
        return " | ".join(profile_parts)
    except Exception as e:
        print(f"Error creating profile for {candidate.get('name', 'Unknown')}: {e}")
        return ""


In [54]:
def generate_all_profile_texts(candidates_data):
    """Generate profile texts for all candidates"""
    profile_texts = []
    candidate_ids = []
    
    for candidate in candidates_data:
        profile_text = create_candidate_profile_text(candidate)
        if profile_text:
            profile_texts.append(profile_text)
            candidate_ids.append(candidate['resumeid'])
    
    return profile_texts, candidate_ids


In [55]:
# Test Phase 2
print("\nPhase 2: Testing profile text generation")
if candidates_data:
    profile_texts, candidate_ids = generate_all_profile_texts(candidates_data)
    print(f"Generated {len(profile_texts)} profile texts")
    print(f"Sample profile text (first 200 chars): {profile_texts[0][:200]}..." if profile_texts else "No profiles generated")



Phase 2: Testing profile text generation
Generated 5 profile texts
Sample profile text (first 200 chars): Name: Rahul Sharma | Preferred Role: Backend Developer | Location: Mumbai, Maharashtra, India | Education: Bachelor of Science in Computer Science from IIT Delhi | Academic Grade: 8.5 | Current/Recent...


In [56]:
profile_texts

['Name: Rahul Sharma | Preferred Role: Backend Developer | Location: Mumbai, Maharashtra, India | Education: Bachelor of Science in Computer Science from IIT Delhi | Academic Grade: 8.5 | Current/Recent Role: Software Engineer at Tata Consultancy Services | Work Location: Mumbai, India | Work Description: Worked on backend systems and REST APIs using Java and Spring Boot. | Technical Skills: Java,Spring Boot,REST APIs,SQL,AWS,Docker,Microservices | Work Availability: Full-time | Full-time Status: Available | Expected Salary: 5000 USD | Resume Content: Software Engineer with 3+ years experience in Java Spring Boot microservices REST APIs backend development MySQL PostgreSQL Docker Kubernetes AWS cloud deployment CI/CD Jenkins Git version control agile methodologies | Professional Summary: Experienced backend developer with 3+ years at TCS, specializing in Java Spring Boot microservices and cloud deployment | Additional Notes: Strong backend developer with TCS experience, excellent probl

In [57]:
# Install required packages first
# !pip install sentence-transformers numpy


In [58]:
import numpy as np
from sentence_transformers import SentenceTransformer

In [59]:

def setup_embedding_model():
    """Initialize the sentence transformer model"""
    try:

        model = SentenceTransformer('all-MiniLM-L6-v2')
        print("Embedding model loaded successfully")
        return model
    except ImportError:
        print("Please install sentence-transformers: pip install sentence-transformers")
        return None
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def generate_embeddings(model, profile_texts):
    """Generate embeddings for profile texts"""
    if not model:
        return None
    try:
        embeddings = model.encode(profile_texts, convert_to_tensor=False)
        return np.array(embeddings, dtype=np.float32)
    except Exception as e:
        print(f"Error generating embeddings: {e}")
        return None


In [60]:

# Test Phase 3
print("\nPhase 3: Testing embedding generation")
embedding_model = setup_embedding_model()
if embedding_model and profile_texts:
    embeddings = generate_embeddings(embedding_model, profile_texts)
    if embeddings is not None:
        print(f"Generated embeddings shape: {embeddings.shape}")
        print(f"Embedding dimension: {embeddings.shape[1]}")
    else:
        print("Failed to generate embeddings")



Phase 3: Testing embedding generation
Embedding model loaded successfully
Generated embeddings shape: (5, 384)
Embedding dimension: 384


In [61]:
len(embeddings),len(embeddings[0])

(5, 384)

### Metadata extraction from query:_- 

In [62]:
from datetime import datetime

def extract_candidate_metadata(candidate):
    """Extract structured metadata for filtering"""
    try:
        # Parse location
        if isinstance(candidate['location'], str):
            location = json.loads(candidate['location'])
        else:
            location = candidate['location']
        
        # Calculate experience years
        work_start = datetime.strptime(str(candidate['work_startdate']), '%Y-%m-%d')
        work_end = datetime.strptime(str(candidate['work_enddate']), '%Y-%m-%d')
        experience_years = (work_end - work_start).days / 365.25
        
        # Extract skills list
        skills_list = [skill.strip() for skill in candidate['skills'].split(',')]
        primary_skills = skills_list[:3]  # First 3 as primary
        
        # Determine company type
        big_companies = ['Google', 'Microsoft', 'Amazon', 'Apple', 'Meta', 'Netflix', 'Tesla']
        company_type = 'MNC' if candidate['company'] in big_companies else 'Other'
        
        metadata = {
            "candidate_id": candidate['resumeid'],
            "name": candidate['name'],
            "preferred_role": candidate['preferredrole'],
            "skills": skills_list,
            "primary_skills": primary_skills,
            "company": candidate['company'],
            "company_type": company_type,
            "location_country": location.get('country', ''),
            "location_city": location.get('city', ''),
            "work_availability": candidate['workavailability'],
            "salary_usd": float(candidate['fulltimesalary']) if candidate['fulltimesalary'] else 0,
            "salary_currency": candidate['fulltimesalarycurrency'],
            "experience_years": round(experience_years, 1),
            "education_level": candidate['degree'],
            "is_remote_available": 'remote' in candidate['workavailability'].lower(),
            "is_active": candidate['fulltimestatus'] == 'Available',
            "last_updated": datetime.now().isoformat()
        }
        
        return metadata
    except Exception as e:
        print(f"Error extracting metadata for {candidate.get('name', 'Unknown')}: {e}")
        return None


In [63]:

def generate_all_metadata(candidates_data):
    """Generate metadata for all candidates"""
    metadata_list = []
    
    for candidate in candidates_data:
        metadata = extract_candidate_metadata(candidate)
        if metadata:
            metadata_list.append(metadata)
    
    return metadata_list



In [64]:
# Test Phase 4
print("\nPhase 4: Testing metadata extraction")
if candidates_data:
    metadata_list = generate_all_metadata(candidates_data)
    print(f"Generated metadata for {len(metadata_list)} candidates")
    if metadata_list:
        print("Sample metadata:")
        for key, value in list(metadata_list[0].items())[:5]:
            print(f"  {key}: {value}")



Phase 4: Testing metadata extraction
Generated metadata for 5 candidates
Sample metadata:
  candidate_id: CAND001
  name: Rahul Sharma
  preferred_role: Backend Developer
  skills: ['Java', 'Spring Boot', 'REST APIs', 'SQL', 'AWS', 'Docker', 'Microservices']
  primary_skills: ['Java', 'Spring Boot', 'REST APIs']


In [65]:
metadata_list[0]

{'candidate_id': 'CAND001',
 'name': 'Rahul Sharma',
 'preferred_role': 'Backend Developer',
 'skills': ['Java',
  'Spring Boot',
  'REST APIs',
  'SQL',
  'AWS',
  'Docker',
  'Microservices'],
 'primary_skills': ['Java', 'Spring Boot', 'REST APIs'],
 'company': 'Tata Consultancy Services',
 'company_type': 'Other',
 'location_country': 'India',
 'location_city': 'Mumbai',
 'work_availability': 'Full-time',
 'salary_usd': 5000.0,
 'salary_currency': 'USD',
 'experience_years': 2.4,
 'education_level': 'Bachelor of Science',
 'is_remote_available': False,
 'is_active': True,
 'last_updated': '2025-06-22T14:40:24.163779'}

In [66]:
for x in metadata_list:
    print(x['preferred_role'])

Backend Developer
Data Analyst
Full Stack Developer
Product Manager
Business Analyst


In [67]:
### phase 5:

In [68]:
def create_vector_storage_structure(embeddings, metadata_list, candidate_ids):
    """Create a structure to store vectors with metadata"""
    vector_storage = []
    
    for i, (embedding, metadata, candidate_id) in enumerate(zip(embeddings, metadata_list, candidate_ids)):
        vector_entry = {
            "id": candidate_id,
            "vector": embedding.tolist(),
            "metadata": metadata
        }
        vector_storage.append(vector_entry)
    
    return vector_storage


In [69]:
def save_vector_storage(vector_storage, filename="candidate_vectors.json"):
    """Save vector storage to file"""
    try:
        import json
        with open(filename, 'w') as f:
            json.dump(vector_storage, f, indent=2)
        print(f"Vector storage saved to {filename}")
        return True
    except Exception as e:
        print(f"Error saving vector storage: {e}")
        return False


In [70]:
# Test Phase 5
print("\nPhase 5: Testing vector storage setup")
if embeddings is not None and metadata_list and candidate_ids:
    vector_storage = create_vector_storage_structure(embeddings, metadata_list, candidate_ids)
    print(f"Created vector storage with {len(vector_storage)} entries")
    
    # Save to file
    save_success = save_vector_storage(vector_storage)
    if save_success:
        print("Vector storage saved successfully")



Phase 5: Testing vector storage setup
Created vector storage with 5 entries
Vector storage saved to candidate_vectors.json
Vector storage saved successfully


In [71]:
## Query analysis:

In [72]:
## NER: named entitiy recognition.

In [73]:
# !pip install spacy 
# !pip install nltk 
# !python -m spacy download en_core_web_sm


In [74]:
import spacy
nlp = spacy.load("en_core_web_sm")  # Or a more robust model
def extract_entities(user_query: str):
    doc = nlp(user_query)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities


In [75]:
query = """I need a Python developer with 4 years of experience",
        "Find me a full-time candidate with remote availability, having worked at big MNC",
        "An Indian origin C++ developer with max salary 500 USD",
        "Java Spring Boot developer with machine learning experience"
 """
extract_entities(query)

[('4 years', 'DATE'),
 ('MNC', 'ORG'),
 ('Indian', 'NORP'),
 ('C++', 'NORP'),
 ('max', 'PERSON'),
 ('500', 'CARDINAL'),
 ('USD', 'PRODUCT'),
 ('Java Spring Boot', 'WORK_OF_ART')]

In [76]:
## SAMPLE QUERY FILTERATION:


In [77]:
import re
from typing import Dict, List

def analyze_query(user_query: str) -> Dict:
    """Analyze user query to extract structured components"""
    analysis = {
        'skills': [],
        'experience_years': None,
        'max_salary': None,
        'currency': 'USD',
        'location': None,
        'work_type': None,
        'company_type': None,
        'semantic_content': user_query.lower()
    }
    
    # Extract skills
    skill_patterns = {
        'python': r'\bpython\b',
        'java': r'\bjava\b',
        'javascript': r'\bjavascript\b|\bjs\b',
        'react': r'\breact\b',
        'node': r'\bnode\.?js\b',
        'sql': r'\bsql\b',
        'aws': r'\baws\b',
        'spring boot': r'\bspring\s*boot\b',
        'machine learning': r'\bmachine\s*learning\b|\bml\b',
        'data analysis': r'\bdata\s*analysis\b'
    }
    
    for skill, pattern in skill_patterns.items():
        if re.search(pattern, user_query.lower()):
            analysis['skills'].append(skill)
    
    # Extract experience
    exp_match = re.search(r'(\d+)\s*(?:years?|yrs?)\s*(?:of\s*)?experience', user_query.lower())
    if exp_match:
        analysis['experience_years'] = int(exp_match.group(1))
    
    # Extract salary
    salary_match = re.search(r'(?:max|maximum|budget|salary).*?(\d+)\s*(usd|dollars?)', user_query.lower())
    if salary_match:
        analysis['max_salary'] = int(salary_match.group(1))
    
    # Extract location
    if 'indian' in user_query.lower() or 'india' in user_query.lower():
        analysis['location'] = 'India'
    
    # Extract work type
    if re.search(r'full.?time', user_query.lower()):
        analysis['work_type'] = 'Full-time'
    elif re.search(r'part.?time', user_query.lower()):
        analysis['work_type'] = 'Part-time'
    
    # Extract company type
    if any(term in user_query.lower() for term in ['big', 'mnc', 'large', 'enterprise']):
        analysis['company_type'] = 'MNC'
    
    return analysis


In [51]:
def test_query_analysis():
    """Test query analysis with sample queries"""
    test_queries = [
        "I need a Python developer with 4 years of experience",
        "Find me a full-time candidate with remote availability, having worked at big MNC",
        "An Indian origin C++ developer with max salary 500 USD",
        "Java Spring Boot developer with machine learning experience"
    ]
    
    for query in test_queries:
        analysis = analyze_query(query)
        print(f"\nQuery: {query}")
        print(f"Analysis: {analysis}")

# Test Phase 6
print("\nPhase 6: Testing query analysis")
test_query_analysis()



Phase 6: Testing query analysis

Query: I need a Python developer with 4 years of experience
Analysis: {'skills': ['python'], 'experience_years': 4, 'max_salary': None, 'currency': 'USD', 'location': None, 'work_type': None, 'company_type': None, 'semantic_content': 'i need a python developer with 4 years of experience'}

Query: Find me a full-time candidate with remote availability, having worked at big MNC
Analysis: {'skills': [], 'experience_years': None, 'max_salary': None, 'currency': 'USD', 'location': None, 'work_type': 'Full-time', 'company_type': 'MNC', 'semantic_content': 'find me a full-time candidate with remote availability, having worked at big mnc'}

Query: An Indian origin C++ developer with max salary 500 USD
Analysis: {'skills': [], 'experience_years': None, 'max_salary': 500, 'currency': 'USD', 'location': 'India', 'work_type': None, 'company_type': None, 'semantic_content': 'an indian origin c++ developer with max salary 500 usd'}

Query: Java Spring Boot developer

In [79]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def filter_candidates_by_metadata(vector_storage, query_analysis):
    """Filter candidates based on metadata criteria"""
    filtered_candidates = []
    
    for candidate in vector_storage:
        metadata = candidate['metadata']
        
        # Check salary constraint
        if query_analysis.get('max_salary'):
            if metadata['salary_usd'] > query_analysis['max_salary']:
                continue
        
        # Check skills
        if query_analysis.get('skills'):
            candidate_skills = [skill.lower() for skill in metadata['skills']]
            required_skills = [skill.lower() for skill in query_analysis['skills']]
            if not any(skill in candidate_skills for skill in required_skills):
                continue
        
        # Check location
        if query_analysis.get('location'):
            if query_analysis['location'].lower() not in metadata['location_country'].lower():
                continue
        
        # Check work type
        if query_analysis.get('work_type'):
            if query_analysis['work_type'].lower() not in metadata['work_availability'].lower():
                continue
        
        # Check company type
        if query_analysis.get('company_type'):
            if metadata['company_type'] != query_analysis['company_type']:
                continue
        
        filtered_candidates.append(candidate)
    
    return filtered_candidates

def semantic_search(query, filtered_candidates, embedding_model, top_k=5):
    """Perform semantic search on filtered candidates"""
    if not filtered_candidates:
        return []
    
    # Generate query embedding
    query_embedding = embedding_model.encode([query], convert_to_tensor=False)[0]
    
    # Get candidate embeddings
    candidate_embeddings = np.array([candidate['vector'] for candidate in filtered_candidates])
    
    # Calculate similarities
    similarities = cosine_similarity([query_embedding], candidate_embeddings)[0]
    
    # Get top-k results
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        results.append({
            'candidate': filtered_candidates[idx],
            'similarity_score': float(similarities[idx])
        })
    
    return results

def hybrid_search(user_query, vector_storage, embedding_model, top_k=5):
    """Perform hybrid search combining filtering and semantic search"""
    # Analyze query
    query_analysis = analyze_query(user_query)
    print(f"Query analysis: {query_analysis}")
    
    # Filter by metadata
    filtered_candidates = filter_candidates_by_metadata(vector_storage, query_analysis)
    print(f"Filtered to {len(filtered_candidates)} candidates")
    
    # Semantic search
    results = semantic_search(user_query, filtered_candidates, embedding_model, top_k)
    
    return results


In [96]:

# Test Phase 7
print("\nPhase 7: Testing similarity search")
if vector_storage and embedding_model:
    test_query = "Candidate with experience in SQL"
    search_results = hybrid_search(test_query, vector_storage, embedding_model, top_k=3)
    
    print(f"\nSearch results for: '{test_query}'")
    # for i, result in enumerate(search_results, 1):
    for  result in search_results:
        candidate = result['candidate']
        score = result['similarity_score']
        print(f"{i}. {candidate['metadata']['name']} - {candidate['metadata']['preferred_role']}")
        print(f"   Similarity: {score:.3f}")
        print(f"   Skills: {', '.join(candidate['metadata']['primary_skills'])}")
        print(f"   Company: {candidate['metadata']['company']}")
        print()



Phase 7: Testing similarity search
Query analysis: {'skills': ['sql'], 'experience_years': None, 'max_salary': None, 'currency': 'USD', 'location': None, 'work_type': None, 'company_type': None, 'semantic_content': 'candidate with experience in sql'}
Filtered to 4 candidates

Search results for: 'Candidate with experience in SQL'
1. Rahul Sharma - Backend Developer
   Similarity: 0.440
   Skills: Java, Spring Boot, REST APIs
   Company: Tata Consultancy Services

1. Priya Patel - Data Analyst
   Similarity: 0.406
   Skills: Python, Data Analysis, SQL
   Company: Google

1. Amit Kumar - Full Stack Developer
   Similarity: 0.368
   Skills: JavaScript, React, Node.js
   Company: Infosys

